## Setup and Installation

Installing required packages for OpenGL rendering.

In [ ]:
# Install required packages
!pip install -q PyOpenGL PyOpenGL_accelerate
!pip install -q glfw
!pip install -q numpy pillow
!pip install -q matplotlib

print("✅ All packages installed successfully!")

## Import Libraries

In [ ]:
import numpy as np
import glfw
from OpenGL.GL import *
from OpenGL.GL.shaders import compileProgram, compileShader
import math
from PIL import Image
import io
import matplotlib.pyplot as plt
from datetime import datetime

print("✅ Libraries imported successfully!")

## Utility Functions

In [ ]:
def create_perspective_matrix(fov, aspect, near, far):
    """Create perspective projection matrix"""
    f = 1.0 / math.tan(math.radians(fov) / 2.0)
    return np.array([
        [f/aspect, 0, 0, 0],
        [0, f, 0, 0],
        [0, 0, (far+near)/(near-far), (2*far*near)/(near-far)],
        [0, 0, -1, 0]
    ], dtype=np.float32)

def create_view_matrix(eye, center, up):
    """Create view matrix (lookAt)"""
    eye = np.array(eye, dtype=np.float32)
    center = np.array(center, dtype=np.float32)
    up = np.array(up, dtype=np.float32)
    
    f = center - eye
    f = f / np.linalg.norm(f)
    
    s = np.cross(f, up)
    s = s / np.linalg.norm(s)
    
    u = np.cross(s, f)
    
    result = np.identity(4, dtype=np.float32)
    result[0, :3] = s
    result[1, :3] = u
    result[2, :3] = -f
    result[0, 3] = -np.dot(s, eye)
    result[1, 3] = -np.dot(u, eye)
    result[2, 3] = np.dot(f, eye)
    
    return result

def create_rotation_matrix(angle, axis):
    """Create rotation matrix around axis"""
    c = math.cos(math.radians(angle))
    s = math.sin(math.radians(angle))
    
    if axis == 'x':
        return np.array([
            [1, 0, 0, 0],
            [0, c, -s, 0],
            [0, s, c, 0],
            [0, 0, 0, 1]
        ], dtype=np.float32)
    elif axis == 'y':
        return np.array([
            [c, 0, s, 0],
            [0, 1, 0, 0],
            [-s, 0, c, 0],
            [0, 0, 0, 1]
        ], dtype=np.float32)
    else:  # z
        return np.array([
            [c, -s, 0, 0],
            [s, c, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ], dtype=np.float32)

def capture_framebuffer(width, height):
    """Capture current framebuffer as PIL Image"""
    pixels = glReadPixels(0, 0, width, height, GL_RGB, GL_UNSIGNED_BYTE)
    image = Image.frombytes('RGB', (width, height), pixels)
    image = image.transpose(Image.FLIP_TOP_BOTTOM)
    return image

print("✅ Utility functions defined!")

## Geometry Generation

In [ ]:
def create_cube():
    """Create cube with positions and normals"""
    vertices = np.array([
        # Front face
        -1, -1,  1,  0,  0,  1,
         1, -1,  1,  0,  0,  1,
         1,  1,  1,  0,  0,  1,
        -1,  1,  1,  0,  0,  1,
        # Back face
        -1, -1, -1,  0,  0, -1,
        -1,  1, -1,  0,  0, -1,
         1,  1, -1,  0,  0, -1,
         1, -1, -1,  0,  0, -1,
        # Top face
        -1,  1, -1,  0,  1,  0,
        -1,  1,  1,  0,  1,  0,
         1,  1,  1,  0,  1,  0,
         1,  1, -1,  0,  1,  0,
        # Bottom face
        -1, -1, -1,  0, -1,  0,
         1, -1, -1,  0, -1,  0,
         1, -1,  1,  0, -1,  0,
        -1, -1,  1,  0, -1,  0,
        # Right face
         1, -1, -1,  1,  0,  0,
         1,  1, -1,  1,  0,  0,
         1,  1,  1,  1,  0,  0,
         1, -1,  1,  1,  0,  0,
        # Left face
        -1, -1, -1, -1,  0,  0,
        -1, -1,  1, -1,  0,  0,
        -1,  1,  1, -1,  0,  0,
        -1,  1, -1, -1,  0,  0,
    ], dtype=np.float32)
    
    indices = np.array([
        0, 1, 2, 2, 3, 0,      # Front
        4, 5, 6, 6, 7, 4,      # Back
        8, 9, 10, 10, 11, 8,   # Top
        12, 13, 14, 14, 15, 12,# Bottom
        16, 17, 18, 18, 19, 16,# Right
        20, 21, 22, 22, 23, 20 # Left
    ], dtype=np.uint32)
    
    return vertices, indices

def create_sphere(radius=1.0, slices=32, stacks=16):
    """Create sphere with positions and normals"""
    vertices = []
    indices = []
    
    for i in range(stacks + 1):
        phi = math.pi * i / stacks
        for j in range(slices + 1):
            theta = 2 * math.pi * j / slices
            
            x = radius * math.sin(phi) * math.cos(theta)
            y = radius * math.cos(phi)
            z = radius * math.sin(phi) * math.sin(theta)
            
            # Position
            vertices.extend([x, y, z])
            # Normal (for sphere, normal = normalized position)
            nx, ny, nz = x/radius, y/radius, z/radius
            vertices.extend([nx, ny, nz])
    
    # Generate indices
    for i in range(stacks):
        for j in range(slices):
            first = i * (slices + 1) + j
            second = first + slices + 1
            
            indices.extend([first, second, first + 1])
            indices.extend([second, second + 1, first + 1])
    
    return np.array(vertices, dtype=np.float32), np.array(indices, dtype=np.uint32)

print("✅ Geometry generation functions defined!")

## Shader Programs

### Gouraud Shading (Per-Vertex Lighting)
Lighting is computed at vertices and interpolated across fragments.

In [ ]:
# Gouraud Shading Vertex Shader (Lighting computed here)
gouraud_vertex_shader = """
#version 330 core

layout(location = 0) in vec3 aPos;
layout(location = 1) in vec3 aNormal;

uniform mat4 model;
uniform mat4 view;
uniform mat4 projection;
uniform mat3 normalMatrix;

uniform vec3 lightPos;
uniform vec3 viewPos;
uniform vec3 lightColor;
uniform vec3 objectColor;
uniform float ambientStrength;
uniform float diffuseStrength;
uniform float specularStrength;
uniform float shininess;

out vec3 vertexColor;

void main()
{
    vec3 fragPos = vec3(model * vec4(aPos, 1.0));
    vec3 normal = normalize(normalMatrix * aNormal);
    
    // Ambient
    vec3 ambient = ambientStrength * lightColor;
    
    // Diffuse (Lambert's cosine law)
    vec3 lightDir = normalize(lightPos - fragPos);
    float diff = max(dot(normal, lightDir), 0.0);
    vec3 diffuse = diffuseStrength * diff * lightColor;
    
    // Specular (Phong reflection model)
    vec3 viewDir = normalize(viewPos - fragPos);
    vec3 reflectDir = reflect(-lightDir, normal);
    float spec = pow(max(dot(viewDir, reflectDir), 0.0), shininess);
    vec3 specular = specularStrength * spec * lightColor;
    
    // Combine all components
    vertexColor = (ambient + diffuse + specular) * objectColor;
    
    gl_Position = projection * view * model * vec4(aPos, 1.0);
}
"""

# Gouraud Shading Fragment Shader (Just interpolates color)
gouraud_fragment_shader = """
#version 330 core

in vec3 vertexColor;
out vec4 FragColor;

void main()
{
    FragColor = vec4(vertexColor, 1.0);
}
"""

print("✅ Gouraud shaders defined!")

### Phong Shading (Per-Fragment Lighting)
Normals are interpolated and lighting is computed for each fragment.

In [ ]:
# Phong Shading Vertex Shader (Passes data to fragment shader)
phong_vertex_shader = """
#version 330 core

layout(location = 0) in vec3 aPos;
layout(location = 1) in vec3 aNormal;

uniform mat4 model;
uniform mat4 view;
uniform mat4 projection;
uniform mat3 normalMatrix;

out vec3 FragPos;
out vec3 Normal;

void main()
{
    FragPos = vec3(model * vec4(aPos, 1.0));
    Normal = normalMatrix * aNormal;
    
    gl_Position = projection * view * model * vec4(aPos, 1.0);
}
"""

# Phong Shading Fragment Shader (Lighting computed here)
phong_fragment_shader = """
#version 330 core

in vec3 FragPos;
in vec3 Normal;

uniform vec3 lightPos;
uniform vec3 viewPos;
uniform vec3 lightColor;
uniform vec3 objectColor;
uniform float ambientStrength;
uniform float diffuseStrength;
uniform float specularStrength;
uniform float shininess;

out vec4 FragColor;

void main()
{
    vec3 normal = normalize(Normal);
    
    // Ambient
    vec3 ambient = ambientStrength * lightColor;
    
    // Diffuse (Lambert's cosine law)
    vec3 lightDir = normalize(lightPos - FragPos);
    float diff = max(dot(normal, lightDir), 0.0);
    vec3 diffuse = diffuseStrength * diff * lightColor;
    
    // Specular (Phong reflection model)
    vec3 viewDir = normalize(viewPos - FragPos);
    vec3 reflectDir = reflect(-lightDir, normal);
    float spec = pow(max(dot(viewDir, reflectDir), 0.0), shininess);
    vec3 specular = specularStrength * spec * lightColor;
    
    // Combine all components
    vec3 result = (ambient + diffuse + specular) * objectColor;
    FragColor = vec4(result, 1.0);
}
"""

print("✅ Phong shaders defined!")

## Rendering Engine Class

In [ ]:
class OpenGLRenderer:
    def __init__(self, width=800, height=600, title="OpenGL Renderer"):
        self.width = width
        self.height = height
        self.window = None
        self.shader_program = None
        self.vao = None
        self.vbo = None
        self.ebo = None
        self.num_indices = 0
        
        # Initialize GLFW
        if not glfw.init():
            raise Exception("GLFW initialization failed")
        
        # Set OpenGL version
        glfw.window_hint(glfw.CONTEXT_VERSION_MAJOR, 3)
        glfw.window_hint(glfw.CONTEXT_VERSION_MINOR, 3)
        glfw.window_hint(glfw.OPENGL_PROFILE, glfw.OPENGL_CORE_PROFILE)
        glfw.window_hint(glfw.VISIBLE, glfw.FALSE)  # Hidden window for rendering
        
        # Create window
        self.window = glfw.create_window(width, height, title, None, None)
        if not self.window:
            glfw.terminate()
            raise Exception("Window creation failed")
        
        glfw.make_context_current(self.window)
        
        # Enable depth testing
        glEnable(GL_DEPTH_TEST)
        
    def load_geometry(self, vertices, indices):
        """Load geometry into GPU buffers"""
        self.num_indices = len(indices)
        
        # Create VAO
        self.vao = glGenVertexArrays(1)
        glBindVertexArray(self.vao)
        
        # Create VBO
        self.vbo = glGenBuffers(1)
        glBindBuffer(GL_ARRAY_BUFFER, self.vbo)
        glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
        
        # Create EBO
        self.ebo = glGenBuffers(1)
        glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, self.ebo)
        glBufferData(GL_ELEMENT_ARRAY_BUFFER, indices.nbytes, indices, GL_STATIC_DRAW)
        
        # Position attribute
        glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 6 * 4, ctypes.c_void_p(0))
        glEnableVertexAttribArray(0)
        
        # Normal attribute
        glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 6 * 4, ctypes.c_void_p(3 * 4))
        glEnableVertexAttribArray(1)
        
        glBindVertexArray(0)
    
    def compile_shader(self, vertex_src, fragment_src):
        """Compile shader program"""
        self.shader_program = compileProgram(
            compileShader(vertex_src, GL_VERTEX_SHADER),
            compileShader(fragment_src, GL_FRAGMENT_SHADER)
        )
    
    def render(self, rotation_angle=0, light_pos=(2, 2, 2), shininess=32, 
               ambient=0.2, diffuse=0.8, specular=1.0):
        """Render the scene"""
        glClearColor(0.1, 0.1, 0.15, 1.0)
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        
        glUseProgram(self.shader_program)
        
        # Create matrices
        model = create_rotation_matrix(rotation_angle, 'y') @ create_rotation_matrix(rotation_angle * 0.7, 'x')
        view = create_view_matrix([0, 0, 6], [0, 0, 0], [0, 1, 0])
        projection = create_perspective_matrix(45, self.width/self.height, 0.1, 100)
        
        # Normal matrix (for transforming normals)
        normal_matrix = np.linalg.inv(model[:3, :3]).T
        
        # Set uniforms
        glUniformMatrix4fv(glGetUniformLocation(self.shader_program, "model"), 1, GL_TRUE, model)
        glUniformMatrix4fv(glGetUniformLocation(self.shader_program, "view"), 1, GL_TRUE, view)
        glUniformMatrix4fv(glGetUniformLocation(self.shader_program, "projection"), 1, GL_TRUE, projection)
        glUniformMatrix3fv(glGetUniformLocation(self.shader_program, "normalMatrix"), 1, GL_TRUE, normal_matrix)
        
        # Lighting parameters
        glUniform3f(glGetUniformLocation(self.shader_program, "lightPos"), *light_pos)
        glUniform3f(glGetUniformLocation(self.shader_program, "viewPos"), 0, 0, 6)
        glUniform3f(glGetUniformLocation(self.shader_program, "lightColor"), 1.0, 1.0, 1.0)
        glUniform3f(glGetUniformLocation(self.shader_program, "objectColor"), 0.3, 0.6, 0.9)
        glUniform1f(glGetUniformLocation(self.shader_program, "ambientStrength"), ambient)
        glUniform1f(glGetUniformLocation(self.shader_program, "diffuseStrength"), diffuse)
        glUniform1f(glGetUniformLocation(self.shader_program, "specularStrength"), specular)
        glUniform1f(glGetUniformLocation(self.shader_program, "shininess"), shininess)
        
        # Draw
        glBindVertexArray(self.vao)
        glDrawElements(GL_TRIANGLES, self.num_indices, GL_UNSIGNED_INT, None)
        glBindVertexArray(0)
        
        glfw.swap_buffers(self.window)
        glfw.poll_events()
    
    def capture(self):
        """Capture current frame as PIL Image"""
        return capture_framebuffer(self.width, self.height)
    
    def cleanup(self):
        """Clean up resources"""
        if self.vao:
            glDeleteVertexArrays(1, [self.vao])
        if self.vbo:
            glDeleteBuffers(1, [self.vbo])
        if self.ebo:
            glDeleteBuffers(1, [self.ebo])
        if self.shader_program:
            glDeleteProgram(self.shader_program)
        if self.window:
            glfw.destroy_window(self.window)
        glfw.terminate()

print("✅ OpenGL Renderer class defined!")

---

## Part A: Render Base Object with Lighting (Ambient + Diffuse)

**Task:** Render a 3D cube with ambient and diffuse lighting only (no specular).  
**Observation:** Surface brightness changes based on object orientation with respect to light.

In [ ]:
# Create renderer
renderer = OpenGLRenderer(800, 600, "Part A: Base Lighting")

# Load cube geometry
vertices, indices = create_cube()
renderer.load_geometry(vertices, indices)

# Compile Phong shader (we'll use Phong for better visuals)
renderer.compile_shader(phong_vertex_shader, phong_fragment_shader)

# Render with ambient + diffuse only (specular = 0)
renderer.render(rotation_angle=30, light_pos=(2, 2, 2), shininess=32,
                ambient=0.2, diffuse=0.8, specular=0.0)

# Capture screenshot
img_part_a = renderer.capture()

# Display
plt.figure(figsize=(10, 8))
plt.imshow(img_part_a)
plt.title('Part A: Ambient + Diffuse Lighting Only', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print("✅ Part A Complete: Ambient and Diffuse lighting applied")
print("   Notice how faces oriented toward the light are brighter")

renderer.cleanup()

---

## Part B: Add Specular Lighting (Phong Reflection)

**Task:** Add specular component using Phong reflection model.  
**Observation:** Highlight appears and moves based on object rotation.

In [ ]:
# Create renderer
renderer = OpenGLRenderer(800, 600, "Part B: Specular Lighting")

# Load cube geometry
vertices, indices = create_cube()
renderer.load_geometry(vertices, indices)

# Compile Phong shader
renderer.compile_shader(phong_vertex_shader, phong_fragment_shader)

# Render with all lighting components
renderer.render(rotation_angle=30, light_pos=(2, 2, 2), shininess=32,
                ambient=0.2, diffuse=0.8, specular=1.0)

# Capture screenshot
img_part_b = renderer.capture()

# Display
plt.figure(figsize=(10, 8))
plt.imshow(img_part_b)
plt.title('Part B: Ambient + Diffuse + Specular Lighting', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print("✅ Part B Complete: Specular highlights added")
print("   Notice the bright specular highlights on surfaces facing the viewer")

renderer.cleanup()

---

## Part C: Implement Gouraud Shading

**Task:** Lighting computed per-vertex, then interpolated across fragments.  
**Observation:** Highlights appear weak/washed out, shiny spots are smeared.

In [ ]:
# Create renderer
renderer = OpenGLRenderer(800, 600, "Part C: Gouraud Shading")

# Load sphere for better visualization of shading differences
vertices, indices = create_sphere(radius=1.5, slices=32, stacks=16)
renderer.load_geometry(vertices, indices)

# Compile Gouraud shader
renderer.compile_shader(gouraud_vertex_shader, gouraud_fragment_shader)

# Render
renderer.render(rotation_angle=30, light_pos=(3, 3, 3), shininess=32,
                ambient=0.2, diffuse=0.8, specular=1.0)

# Capture screenshot
img_part_c = renderer.capture()

# Display
plt.figure(figsize=(10, 8))
plt.imshow(img_part_c)
plt.title('Part C: Gouraud Shading (Per-Vertex Lighting)', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print("✅ Part C Complete: Gouraud shading implemented")
print("   Notice: Specular highlights are less sharp and appear washed out")
print("   This is because lighting is only computed at vertices")

renderer.cleanup()

---

## Part D: Implement Phong Shading

**Task:** Lighting computed per-fragment (per-pixel).  
**Observation:** Sharp, realistic highlights with smooth shading.

In [ ]:
# Create renderer
renderer = OpenGLRenderer(800, 600, "Part D: Phong Shading")

# Load sphere
vertices, indices = create_sphere(radius=1.5, slices=32, stacks=16)
renderer.load_geometry(vertices, indices)

# Compile Phong shader
renderer.compile_shader(phong_vertex_shader, phong_fragment_shader)

# Render
renderer.render(rotation_angle=30, light_pos=(3, 3, 3), shininess=32,
                ambient=0.2, diffuse=0.8, specular=1.0)

# Capture screenshot
img_part_d = renderer.capture()

# Display
plt.figure(figsize=(10, 8))
plt.imshow(img_part_d)
plt.title('Part D: Phong Shading (Per-Fragment Lighting)', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print("✅ Part D Complete: Phong shading implemented")
print("   Notice: Specular highlights are sharp and realistic")
print("   Lighting is computed for every pixel, resulting in smooth shading")

renderer.cleanup()

---

## Part E: Compare Gouraud vs Phong (Side-by-Side)

**Task:** Render two identical spheres side-by-side with different shading models.  
**Observation:** Direct comparison shows Phong's superior quality.

In [ ]:
# Display side-by-side comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(img_part_c)
axes[0].set_title('Gouraud Shading (Per-Vertex)', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(img_part_d)
axes[1].set_title('Phong Shading (Per-Fragment)', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("="*70)
print("COMPARISON: GOURAUD vs PHONG SHADING")
print("="*70)
print("\n📊 Gouraud Shading (Left):")
print("   - Lighting computed at vertices only")
print("   - Colors linearly interpolated across surface")
print("   - Specular highlights appear washed out and smeared")
print("   - Less accurate, but faster to compute")
print("   - Good for low-poly models with matte surfaces")

print("\n📊 Phong Shading (Right):")
print("   - Lighting computed at every pixel (fragment)")
print("   - Normals interpolated across surface")
print("   - Sharp, realistic specular highlights")
print("   - Smooth, high-quality shading")
print("   - More computationally expensive")
print("   - Industry standard for realistic rendering")

print("\n✅ Part E Complete: Side-by-side comparison demonstrates")
print("   Phong shading produces significantly better visual quality")

---

## Part F: Change Material Shininess (n factor)

**Task:** Vary shininess exponent to observe highlight changes.  
**Values:** n = 4, 16, 64  
**Observation:** Higher n = sharper highlight, Lower n = blurry highlight.

In [ ]:
shininess_values = [4, 16, 64]
images_shininess = []

for n in shininess_values:
    # Create renderer
    renderer = OpenGLRenderer(600, 600, f"Shininess n={n}")
    
    # Load sphere
    vertices, indices = create_sphere(radius=1.5, slices=32, stacks=16)
    renderer.load_geometry(vertices, indices)
    
    # Compile Phong shader
    renderer.compile_shader(phong_vertex_shader, phong_fragment_shader)
    
    # Render with different shininess
    renderer.render(rotation_angle=30, light_pos=(3, 3, 3), shininess=n,
                    ambient=0.2, diffuse=0.8, specular=1.0)
    
    # Capture
    images_shininess.append(renderer.capture())
    
    renderer.cleanup()

# Display all shininess variations
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (img, n) in enumerate(zip(images_shininess, shininess_values)):
    axes[i].imshow(img)
    axes[i].set_title(f'Shininess n = {n}', fontsize=14, fontweight='bold')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print("="*70)
print("SHININESS ANALYSIS")
print("="*70)
print("\n📊 n = 4 (Low Shininess):")
print("   - Wide, diffuse highlight")
print("   - Represents rough/matte materials (e.g., chalk, unpolished wood)")

print("\n📊 n = 16 (Medium Shininess):")
print("   - Moderate highlight size")
print("   - Represents semi-glossy materials (e.g., plastic, painted surfaces)")

print("\n📊 n = 64 (High Shininess):")
print("   - Sharp, concentrated highlight")
print("   - Represents very glossy materials (e.g., polished metal, glass)")

print("\n✅ Part F Complete: Shininess exponent controls highlight sharpness")

---

## Part G: Change Light Position

**Task:** Move light to different positions and observe changes.  
**Positions:** (2, 2, 2), (-2, 1, 0), (0, 5, 0)  
**Observation:** Brightness on faces changes, highlight location shifts.

In [ ]:
light_positions = [
    (2, 2, 2, "Top-Right-Front"),
    (-2, 1, 0, "Left-Center"),
    (0, 5, 0, "Directly Above")
]

images_lights = []

for pos_x, pos_y, pos_z, label in light_positions:
    # Create renderer
    renderer = OpenGLRenderer(600, 600, f"Light at {label}")
    
    # Load sphere
    vertices, indices = create_sphere(radius=1.5, slices=32, stacks=16)
    renderer.load_geometry(vertices, indices)
    
    # Compile Phong shader
    renderer.compile_shader(phong_vertex_shader, phong_fragment_shader)
    
    # Render with different light position
    renderer.render(rotation_angle=30, light_pos=(pos_x, pos_y, pos_z), shininess=32,
                    ambient=0.2, diffuse=0.8, specular=1.0)
    
    # Capture
    images_lights.append((renderer.capture(), label, (pos_x, pos_y, pos_z)))
    
    renderer.cleanup()

# Display all light positions
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (img, label, pos) in enumerate(images_lights):
    axes[i].imshow(img)
    axes[i].set_title(f'Light at {pos}\n({label})', fontsize=12, fontweight='bold')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print("="*70)
print("LIGHT POSITION ANALYSIS")
print("="*70)
print("\n📊 Light at (2, 2, 2) - Top-Right-Front:")
print("   - Illuminates right and top portions")
print("   - Highlight appears on upper-right area")
print("   - Left and bottom faces are darker")

print("\n📊 Light at (-2, 1, 0) - Left-Center:")
print("   - Illuminates left side of sphere")
print("   - Highlight shifts to left portion")
print("   - Right side falls into shadow")

print("\n📊 Light at (0, 5, 0) - Directly Above:")
print("   - Illuminates top hemisphere")
print("   - Highlight appears at the top")
print("   - Bottom hemisphere is darker")

print("\n✅ Part G Complete: Light position dramatically affects appearance")
print("   Diffuse lighting follows Lambert's cosine law")
print("   Specular highlights move with light and view directions")

---

## Summary and Conclusions

### Key Findings

#### 1. Lighting Components
- **Ambient:** Constant base illumination (simulates indirect light)
- **Diffuse:** Direction-dependent using Lambert's cosine law (angle between normal and light)
- **Specular:** View-dependent using Phong reflection model (creates highlights)

#### 2. Shading Models Comparison

**Gouraud Shading (Per-Vertex):**
- ✅ Faster computation (lighting calculated once per vertex)
- ✅ Suitable for low-poly models with matte surfaces
- ❌ Specular highlights appear washed out
- ❌ Highlights can be missed between vertices
- ❌ Linear color interpolation reduces realism

**Phong Shading (Per-Fragment):**
- ✅ High-quality, realistic results
- ✅ Sharp, accurate specular highlights
- ✅ Smooth shading across surfaces
- ✅ Industry standard for modern rendering
- ❌ More computationally expensive
- ❌ Requires interpolating normals

#### 3. Material Properties (Shininess)
- Low shininess (n=4): Wide, diffuse highlights → rough/matte materials
- Medium shininess (n=16): Moderate highlights → plastic/painted surfaces
- High shininess (n=64): Sharp highlights → polished metal/glass

#### 4. Light Position Effects
- Light position determines which surfaces receive direct illumination
- Specular highlights follow reflection angle (view + light direction)
- Multiple lights can be combined for complex lighting scenarios

### Equations Used

**Phong Reflection Model:**
$$I = I_a + I_d + I_s$$

Where:
- $I_a = k_a \cdot L_a$ (Ambient)
- $I_d = k_d \cdot L_d \cdot (N \cdot L)$ (Diffuse - Lambert)
- $I_s = k_s \cdot L_s \cdot (R \cdot V)^n$ (Specular - Phong)

**Vectors:**
- $N$ = Surface normal (perpendicular to surface)
- $L$ = Light direction (from surface to light)
- $V$ = View direction (from surface to camera)
- $R$ = Reflection vector = $2(N \cdot L)N - L$

### CLO Achievement

✅ **CLO-2:** Applied HCI/CG principles to lighting design  
✅ **CLO-3:** Implemented Gouraud and Phong shading in OpenGL  
✅ **CLO-4:** Analyzed and compared different rendering techniques  

---

## End of Lab Report